In [ ]:
# !pip install langchain langchain-community sentence-transformers langchain_elasticsearch huggingface_hub

In [1]:
import os
from dotenv import load_dotenv

load_dotenv(verbose=True)
ES_CLOUD_ID = os.getenv('ES_CLOUD_ID')
ES_USER = os.getenv('ES_USER')
ES_PASSWORD = os.getenv('ES_PASSWORD')
ES_API_KEY = os.getenv('ES_API_KEY')
OPENAI_KEY = os.getenv('OPENAI_KEY')
index_name = "helloworld9"

In [2]:
import torch

torch.cuda.empty_cache()

In [3]:
from langchain_community.embeddings.openai import OpenAIEmbeddings

def setup_embeddings():
    # Huggingface embedding setup
    print(">> Prep. OpenAI embedding setup")
    return OpenAIEmbeddings(openai_api_key=OPENAI_KEY)

openai = setup_embeddings()

>> Prep. OpenAI embedding setup


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [4]:
# ElasticSearch vectorstore in langchain style
from langchain_elasticsearch import ElasticsearchStore

db = ElasticsearchStore(
    es_cloud_id=ES_CLOUD_ID,
    es_user=ES_USER,
    es_password=ES_PASSWORD,
    es_api_key=ES_API_KEY,
    index_name=index_name,
    embedding=openai,
    #es_url = 'https://bfc3fd6827b94a4ea8d56f9805a44cae.us-central1.gcp.cloud.es.io:443'
)

In [5]:
index_name

'helloworld9'

### Data chunking

In [6]:
import pandas as pd
from tqdm import tqdm

# 임시로 경로 변경
data = pd.read_csv('data/preprocessed.csv')
#data['text'] = ''
data.head()

,source,title,content
0,법무부 외국인 체류관리,외국인 체류관리,category: 외국인 체류관리\ntext:\n법무부는 합법적인 이주를 장려하고 ...
1,법무부 외국인 체류관리,노동시장의 수요를 고려한 외국인유입정책,category: 노동시장의 수요를 고려한 외국인유입정책\ntext:\n법무부는 고...
2,법무부 외국인 체류관리,계절근로자 제도,category: 계절근로자 제도\ntext:\n농‧어촌의 고질적인 인력부족 문제를...
3,법무부 외국인 체류관리,외국인 숙련기능인력 점수제 비자,category: 외국인 숙련기능인력 점수제 비자\ntext:\n주조‧금형‧용접 등...
4,법무부 외국인 체류관리,해외 우수인재 유치,category: 해외 우수인재 유치\ntext:\n국가경쟁력에 직결되는 우수인재‧...


In [7]:
data.shape

(3433, 3)

In [8]:
from transformers import AutoTokenizer
# LLaMA-3 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("MLP-KTLim/llama-3-Korean-Bllossom-8B")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:99: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
from langchain.docstore.document import Document
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

# RAG style : title + [SEP] + contents
# seperation 지정되어 있지 않음 -> 공백으로 처리
def make_chunk_data(titles: list, contents: list) -> list:
    if tokenizer.sep_token != None:
        print('## seperated by : ', tokenizer.sep_token)
        chunk_data = [title + tokenizer.sep_token + content for title, content in zip(titles, contents)]
    else:
        chunk_data = [title + " " + content for title, content in zip(titles, contents)]

    return chunk_data


# 토크나이저 기준 분할
def data_chunking(titles: list, contents: list, sources: list) -> Document:
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        tokenizer,
        chunk_size=500,
        chunk_overlap=100
    )
    chunk_data = make_chunk_data(titles, contents)
    chunks = text_splitter.create_documents(chunk_data, sources)

    return chunks

In [10]:
# 빈 리스트 초기화
titles = []
contents = []
sources = []

# 데이터프레임 순회
for index, row in data.iterrows():
    titles.append(row['title'])
    contents.append(row['content'])
    sources.append({'source': row['source']})

# 결과 확인
#print("Titles:", titles)
#print("Contents:", contents)

In [11]:
len(titles), len(contents), len(sources)

(3433, 3433, 3433)

In [12]:
chunks = data_chunking(titles, contents, sources)

In [13]:
len(chunks)

6078

In [14]:



# # RecursiveCharacterTextSplitter 초기화
# text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
#     tokenizer,
#     chunk_size=512,
#     chunk_overlap=100
# )
# chunk_data = make_chunk_data(titles,conents)
# chunks = text_splitter.create_douments(shunk_data)


# # 새로운 DataFrame을 저장할 리스트
# new_rows = []

# # 각 행을 순회하면서 처리
# for _, row in data.iterrows():
#     content = row['content']
#     chunks = text_splitter.split_text(content)

#     # 각 청크에 대해 새로운 행 생성
#     for chunk in chunks:
#         new_row = row.copy()  # 기존 행의 모든 데이터 복사
#         new_row['text'] = chunk  # 'text' 열에 청크 추가
#         new_rows.append(new_row)

# # 새로운 DataFrame 생성
# chunk_data = pd.DataFrame(new_rows)

# # 'content' 열 제거 (선택사항)
# chunk_data = chunk_data.drop(columns=['content'])

# # 인덱스 재설정
# chunk_data = chunk_data.reset_index(drop=True)

# # 결과 확인
# print(chunk_data.head())
# print(f"Total rows in new DataFrame: {len(chunk_data)}")

In [15]:
# data.shape, chunk_data.shape

In [16]:
# chunk_data.head()

In [17]:
# batchtext = list(chunk_data['text'])
# len(batchtext)

In [18]:
chunks[0:2]

[Document(metadata={'source': '법무부 외국인 체류관리'}, page_content='외국인 체류관리 category: 외국인 체류관리\ntext:\n법무부는 합법적인 이주를 장려하고 촉진하며 지원하기 위하여 다양한 정책적 지원방안을 시행하고 있습니다.\n2013년부터 교수/연구원 등 전문인력, 의료관광객, 단체관광객 등 일부 외국인의 경우, 재외공관을 방문하지 않고도 온라인(비자포털, www.visa.go.kr)으로 대한민국 비자를 신청/발급 받아 신속, 편리하게 입국할 수 있도록 전자비자 제도를 시행함으로써 비자신청 민원편의를 도모하고 있습니다.\n또한 유학비자를 발급받고자 하는 학생들의 경우, 우수한 학습프로그램과 유학생 지원체계를 보유하고 있는 학교를 선택하도록 장려함으로써 우수한 외국 학생들이 한국에서 인적 자본을 성장시킬 수 있도록 지속적으로 지원하고 있습니다.'),
 Document(metadata={'source': '법무부 외국인 체류관리'}, page_content='노동시장의 수요를 고려한 외국인유입정책 category: 노동시장의 수요를 고려한 외국인유입정책\ntext:\n법무부는 고용노동부, 산업통상자원부 등 관계부처와 유기적으로 협력하여 외국인근로자 정책을 수립･시행하고 있습니다.\n농어업 부문 등 1차 산업 및 도장･금형･주조 등 기초 2차 산업의 비숙련 부문 등 인력부족을 겪고 있는 영역에서 외국 인력이 활용될 수 있도록 다양한 비자정책을 시행하고 있습니다.')]

In [19]:
len(chunks)

6078

In [20]:
index_name

'helloworld9'

In [21]:
# """
# 런타임 모두 실행 시 DB에 데이터가 중복 저장되는 것을 막기 위해 주석 처리 해놨습니다.
# 필요한 코드이니 지우지 말아주세요.
# """
#DB에 텍스트 데이터 추가

#꼭 확인!!
index_name = index_name

db.from_documents(chunks,
              embedding=openai,
              es_cloud_id=ES_CLOUD_ID,
              es_user=ES_USER,
              es_password=ES_PASSWORD,
              es_api_key=ES_API_KEY,
              index_name=index_name)